In [0]:
from google.colab import drive
drive.mount('content')
%cd content/"My Drive"/reddit

Drive already mounted at content; to attempt to forcibly remount, call drive.mount("content", force_remount=True).
/content/content/My Drive/reddit


Importing necessary libraries and reading the cleaned csv file

In [0]:
import pandas as pd
data = pd.read_csv('reddit.csv')
data = data.drop_duplicates(subset='id',keep='last')
data.drop(['author_flair_css_class','author_flair_richtext','author_flair_text','author_flair_type','author_fullname'
           ,'approved_at_utc','banned_at_utc','full_link','id','link_flair_background_color','permalink','media_embed'
           ,'secure_media','secure_media_embed','media','author_flair_background_color','media_metadata','discussion_type'
           ,'crosspost_parent','crosspost_parent_list','author_created_utc','category','content_categories','removal_reason'
           ,'subreddit_name_prefixed','author_id','previous_visits','view_count','thumbnail_height','thumbnail_width'
           ,'subreddit_subscribers','event_end','event_start','created_utc','retrieved_on','created','edited',
           'updated_utc','mod_reports','user_reports','rte_mode','steward_reports','event_is_live','suggested_sort'
           ,'distinguished','can_gild','archived','og_title','og_description','quarantine','author_flair_text_color'
           ,'author_flair_template_id','link_flair_template_id','link_flair_css_class','treatment_tags','subreddit_id'
           ,'subreddit','preview','gildings','awarders','allow_live_comments','all_awardings','link_flair_text_color',
           'link_flair_richtext','link_flair_type','pwls','wls','subreddit_type'],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,16,18,19,20,21,22,24,25,26,27,28,29,30,31,35,37,38,39,41,45,46,47,51,57,59,62,63,64,65,66,67,68,70,71,72,73,74,76,78,79,80,82,84,85,87,88,89,90,95,96,97,98,99,100,101,102,103,107,108) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
data = data[data['link_flair_text'].notna()]

data.link_flair_text = data.link_flair_text.replace('Science', 'Science/Technology')
data.link_flair_text = data.link_flair_text.replace('Politics [Megathread]', 'Politics')
data.link_flair_text = data.link_flair_text.replace('Policy', 'Policy/Economy')
data.link_flair_text = data.link_flair_text.replace('Politics [OLD]', 'Politics')
data.link_flair_text = data.link_flair_text.replace('Technology', 'Science/Technology')
data.link_flair_text = data.link_flair_text.replace('Politics -- Source in comments', 'Politics')
data.link_flair_text = data.link_flair_text.replace('Policy/Economy -2017 Article ', 'Policy/Economy')

data = data[data.link_flair_text.isin(['AskIndia','Non-Political','Scheduled','Photography','Science/Technology',
                                         'Politics','Business/Finance','Policy/Economy','Sports','Food','[R]eddiquette'])]

data.brand_safe.fillna(False,inplace=True)
data.removed_by.fillna('none',inplace=True)
data.gilded.fillna(0.0,inplace=True)
data.loc[(~data.poll_data.isna()),'poll_data']=True
data.poll_data.fillna(False,inplace=True)
data.hidden.fillna(True,inplace=True)
data.banned_by.fillna('none',inplace=True)
data.author_cakeday.fillna(False,inplace=True)
data.whitelist_status.fillna('no_ads',inplace=True)
data.total_awards_received.fillna(0.0,inplace=True)
data.thumbnail.fillna('self',inplace=True)
data.spoiler.fillna(False,inplace=True)
data.send_replies.fillna(True,inplace=True)
data.removed_by_category.fillna('none',inplace=True)
data.post_hint.fillna('text',inplace=True)
data.pinned.fillna(True,inplace=True)
data.parent_whitelist_status.fillna('no_ads',inplace=True)
data.num_crossposts.fillna(0.0,inplace=True)
data.no_follow.fillna(False,inplace=True)
data.media_only.fillna(True,inplace=True)
data.locked.fillna(True,inplace=True)
data.is_video.fillna(False,inplace=True)
data.is_robot_indexable.fillna(True,inplace=True)
data.is_reddit_media_domain.fillna(False,inplace=True)
data.is_original_content.fillna(False,inplace=True)
data.is_meta.fillna(True,inplace=True)
data.is_crosspostable.fillna(True,inplace=True)
data.contest_mode.fillna(False,inplace=True)
data.can_mod_post.fillna(True,inplace=True)
data.author_premium.fillna(False,inplace=True)
data.author_patreon_flair.fillna(True,inplace=True)
data.selftext.fillna('none',inplace=True)

Changing bool values into int values for feeding into neural networks.

In [0]:
data = data*1
data.head()

,author,author_patreon_flair,author_premium,can_mod_post,contest_mode,domain,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,pinned,post_hint,removed_by_category,score,selftext,send_replies,spoiler,stickied,thumbnail,title,total_awards_received,url,whitelist_status,link_flair_text,author_cakeday,banned_by,poll_data,gilded,removed_by,hidden,brand_safe
1,pkmcreation,0,0,0,0,pkmcreation.com,0,0,0,0,0,0,0,0,0,1,0,0.0,0,all_ads,0,text,reddit,1,none,1,0,0,default,North Korea: ऐसा देश जहां बुनियादी अधिकारों को...,0.0,https://www.pkmcreation.com/2020/04/north-kore...,all_ads,Non-Political,0,none,0,0.0,none,1,0
2,BigBulkemails,0,0,0,0,self.india,1,0,0,0,1,1,0,0,0,1,1,0.0,0,all_ads,0,text,none,1,[Article](https://www.india.com/viral/over-93-...,1,0,0,self,93.5% people think Modi govt will handle Covid...,0.0,https://www.reddit.com/r/india/comments/g6jvwx...,all_ads,Politics,0,none,0,0.0,none,1,0
3,Ib90,0,0,0,0,indiatoday.in,1,0,0,0,1,0,0,0,0,1,0,0.0,0,all_ads,0,text,none,1,none,1,0,0,default,"BJP spreading virus of communal hatred, prejud...",0.0,https://www.indiatoday.in/india/story/bjp-spre...,all_ads,Politics,0,none,0,0.0,none,1,0
5,tandoori_idli,0,0,0,0,ndtv.com,1,0,0,0,1,0,0,0,0,1,0,0.0,0,all_ads,0,text,none,1,none,1,0,0,default,Increment On Dearness Allowance For Central Go...,0.0,https://www.ndtv.com/business/covid-19-news-in...,all_ads,Non-Political,0,none,0,0.0,none,1,0
6,thestreetsareours,0,0,0,0,timesofindia.indiatimes.com,1,0,0,0,1,0,0,0,0,1,0,0.0,0,all_ads,0,text,none,1,none,0,0,0,default,6 Muslim families in Jind district convert to ...,0.0,https://timesofindia.indiatimes.com/city/gurga...,all_ads,Non-Political,0,none,0,0.0,none,1,0


Dividing the data into categorical, numerical and textual features for individual preprocessing.

In [0]:
#data.drop(['index'],axis=1,inplace=True)
numeric_features = data[['gilded','score','total_awards_received','num_crossposts','num_comments']]
categorical_features = data[['author_patreon_flair','author_premium','can_mod_post','contest_mode',
                             'is_crosspostable','is_meta','is_original_content','is_reddit_media_domain',
                             'is_robot_indexable','is_self','is_video','locked','media_only','no_follow',
                             'parent_whitelist_status','pinned','post_hint','removed_by_category',
                             'send_replies','stickied','whitelist_status','author_cakeday','thumbnail',
                             'spoiler','locked','over_18','is_original_content','is_self',
                           'poll_data','removed_by','banned_by','hidden','brand_safe']]
text_features = data[['domain','title','url']]
target_feature = data[['link_flair_text']]  

In [0]:
target_feature.shape

(591484, 1)

Mapping the categorical variable's values to an integer, so that it can be feeded to the neural network.

In [0]:

categorical_features['thumbnail'] = categorical_features['thumbnail'].str.replace('http\S+|www.\S+', 'link', case=False)      
thumbnail_labels = categorical_features['thumbnail'].astype('category').cat.categories.tolist()
thumbnail_map_comp = {'default': 1, 'link': 2, 'nsfw': 3, 'self': 4, 'spoiler': 5}
categorical_features.thumbnail = categorical_features.thumbnail.map(thumbnail_map_comp)

whitelist_status_labels = categorical_features['whitelist_status'].astype('category').cat.categories.tolist()
whitelist_status_map_comp = {'all_ads': 1, 'no_ads': 2, 'promo_adult_nsfw': 3}
categorical_features.whitelist_status = categorical_features.whitelist_status.map(whitelist_status_map_comp)

banned_by_labels = categorical_features['banned_by'].astype('category').cat.categories.tolist()
banned_by_map_comp = {'moderators': 1, 'none': 2}
categorical_features.banned_by = categorical_features.banned_by.map(banned_by_map_comp)

parent_whitelist_status_labels = categorical_features['parent_whitelist_status'].astype('category').cat.categories.tolist()
parent_whitelist_status_map_comp = {'all_ads': 1, 'no_ads': 2}
#print(parent_whitelist_status_map_comp)
categorical_features.parent_whitelist_status = categorical_features.parent_whitelist_status.map(parent_whitelist_status_map_comp)

post_hint_labels = categorical_features['post_hint'].astype('category').cat.categories.tolist()
post_hint_map_comp = {'hosted:video': 1, 'image': 2, 'link': 3, 'rich:video': 4, 'self': 5, 'text': 6, 'video': 7}
#print(post_hint_map_comp)
categorical_features.post_hint = categorical_features.post_hint.map(post_hint_map_comp)

removed_by_category_labels = categorical_features['removed_by_category'].astype('category').cat.categories.tolist()
removed_by_category_map_comp = {'automod_filtered': 1, 'deleted': 2, 'moderator': 3, 'none': 4, 'reddit': 5}
#print(removed_by_category_map_comp)
categorical_features.removed_by_category = categorical_features.removed_by_category.map(removed_by_category_map_comp)

removed_by_labels = categorical_features['removed_by'].astype('category').cat.categories.tolist()
removed_by_map_comp = {'reddit': 1, 'deleted': 2, 'moderator': 3, 'none': 4}
#print(removed_by_category_map_comp)
categorical_features.removed_by = categorical_features.removed_by.map(removed_by_map_comp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
categorical_features.head()

,author_patreon_flair,author_premium,can_mod_post,contest_mode,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,locked,media_only,no_follow,parent_whitelist_status,pinned,post_hint,removed_by_category,send_replies,stickied,whitelist_status,author_cakeday,thumbnail,spoiler,locked,over_18,is_original_content,is_self,poll_data,removed_by,banned_by,hidden,brand_safe
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,6,5,1,0,1,0,1,0,0,0,0,0,0,4,2,1,0
2,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,6,4,1,0,1,0,4,0,0,0,0,1,0,4,2,1,0
3,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,6,4,1,0,1,0,1,0,0,0,0,0,0,4,2,1,0
5,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,6,4,1,0,1,0,1,0,0,0,0,0,0,4,2,1,0
6,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,6,4,0,0,1,0,1,0,0,0,0,0,0,4,2,1,0


Using label binarizer to transform the different classes in target column(i.e. link flair type) into vectors.

In [0]:
from sklearn.preprocessing import LabelBinarizer
le = LabelBinarizer()
le.fit(target_feature.values)
label = le.transform(target_feature.values)

Cleaning the text in title column by removing punctuations, converting text to lowercase and removing whitespace.

In [0]:
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

text_features['title'] = text_features['title'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
text_features['title'] = text_features['title'].str.lower()
text_features['title'] = text_features['title'].apply(lambda x:' '.join(x.split()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

Making a tokeniztion process for urls as they may present some information regarding the post and standard libs do not provide tools for such purposes.

We remove '/', '-' , '.', 'html', 'https' and numbers from the urls as they do not provide any information and occur too many times.

In [0]:
def getUrltokens(input):
    tokens_1 = str(input.encode('utf-8')).split('/')
    alls = []
    for i in tokens_1:
        tokens = str(i).split('-')
        dots = []
        for j in range(0,len(tokens)):
            temp = str(tokens[j]).split('.')
            dots = dots + temp
        alls = alls + tokens + dots
    all_tok = list(set(alls))
    if 'html' in all_tok:
        all_tok.remove('html')
    if 'https' in all_tok:
        all_tok.remove('https')    
    return ' '.join(all_tok)
text_features['url'] =  text_features['url'].apply(getUrltokens)
text_features['domain'] =  text_features['domain'].apply(getUrltokens)
text_features['url'] = text_features['url'].str.replace("[0-9]", " ")
text_features['domain'] = text_features['domain'].str.replace("[0-9]", " ")

text_features['url'] = text_features['url'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
text_features['domain'] = text_features['domain'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [0]:
text_features

,domain,title,url
1,b'pkmcreation.com' b'pkmcreation com',north korea ऐसा देश जहां बुनियादी अधिकारों को ...,com pkmcreation www www.pkmcreation.com dicta...
2,b'self.india' india' b'self,93.5 people think modi govt will handle covid1...,india com www ' www.reddit.com reddit r b'htt...
3,in' b'indiatoday.in' b'indiatoday,"bjp spreading virus of communal hatred, prejud...",www.indiatoday.in in www hatred during story ...
5,b'ndtv com' b'ndtv.com',increment on dearness allowance for central go...,www amp ampakamai rumoff' government p...
6,indiatimes com' b'timesofindia b'timesofindia....,6 muslim families in jind district convert to ...,indiatimes in cmsfrommdr' timesofi...
...,...,...,...
1007734,com' b'imgur b'imgur.com',my friend just completed cycling 6200 kms from...,fv DA' imgur com imgur.com b'http
1009505,b'i.imgur.com' b'i imgur com',the two india,imgur com i.imgur.com unguR.jpg' i unguR jpg'...
1011150,b'dailypioneer.com' com' b'dailypioneer,interesting new series on ruskin bond will soo...,dailypioneer com www with html' past.html' ww...
1015894,b'i.imgur.com' b'i imgur com',migration pattern of researchers see india,imgur com i.imgur.com eeSrc i eeSrc.jpg' jpg'...


Converting our feature dataframes into numpy arrays.

In [0]:
import numpy as np
categorical_features = np.asarray(categorical_features).astype(np.float32)
numeric_features = np.asarray(numeric_features).astype(np.float32)
title = np.asarray(text_features.title)
domain = np.asarray(text_features.domain)
url = np.asarray(text_features.url)

[[  0.   1.   0.   0.   0.]
 [  0.   1.   0.   0.   1.]
 [  0.   1.   0.   0.   0.]
 ...
 [  0.   1.   0.   0.   0.]
 [  0. 114.   0.   0.  52.]
 [  0. 109.   0.   0.  33.]]


Importing tensorflow and tensorflow hub which contain the tools for building a model.

The hub contains the universal sentence encoder which can encode multilingual text into vectors, as in our case we have Hindi-English text in some cases.

We also instantiate three such layers each for our chosen textual features.

In [0]:
import tensorflow as tf
import tensorflow_text
from tensorflow import keras
tf.keras.backend.clear_session() 
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM,concatenate
import tensorflow_hub as hub

#embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
#embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
#embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1"
#embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
elmo = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3"

hub_layer1 = hub.KerasLayer(elmo, input_shape=[], 
                           dtype=tf.string, trainable=False)
hub_layer2 = hub.KerasLayer(elmo, input_shape=[], 
                           dtype=tf.string, trainable=False)
hub_layer3 = hub.KerasLayer(elmo, input_shape=[], 
                           dtype=tf.string, trainable=False)

We define the input layers for our model.

In [0]:
input_num = Input(shape=(5,))
input_cat = Input(shape=(33,))
input_title = Input(shape=(), dtype=tf.string)
input_domain = Input(shape=(), dtype=tf.string)
input_url = Input(shape=(), dtype=tf.string)

Define model structure for title.

hub_layer-->dense-->dense

In [0]:
title_feat = hub_layer1(input_title)
title_feat = Dense(128,activation='relu')(title_feat)
title_feat = Dense(32,activation='relu')(title_feat)

Define model structure for domain.

hub_layer-->dense-->dense

In [0]:
domain_feat = hub_layer2(input_domain)
domain_feat = Dense(128,activation='relu')(domain_feat)
domain_feat = Dense(32,activation='relu')(domain_feat)

Define model structure for url.

hub_layer-->dense-->dense

In [0]:
url_feat = hub_layer3(input_url)
url_feat = Dense(128,activation='relu')(url_feat)
url_feat = Dense(32,activation='relu')(url_feat)

Define model structure for numerical features.

dense-->dense-->dense

In [0]:
num_feat = Dense(16,activation='relu')(input_num)
num_feat = Dense(32,activation='relu')(num_feat)
num_feat = Dense(22,activation='relu')(num_feat)

Define model structure for categorical features.

dense-->dense-->dense

In [0]:
cat_feat = Dense(64,activation='relu')(input_cat)
cat_feat = Dense(128,activation='relu')(cat_feat)
cat_feat = Dense(68,activation='relu')(cat_feat)

We concatenate the the features from all the above model structures to produce a unified output.

In [0]:
x = concatenate([title_feat, domain_feat, url_feat, num_feat,cat_feat])


We define the output for our model and set activation as softmax as probabilites of all the categories will sum to 1 and will provide insight into model's behviour.

In [0]:
output = Dense(label.shape[1],activation='softmax')(x)
model = keras.Model(inputs=[input_cat, input_num,input_title,input_domain,input_url], outputs=output)
model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 5)]          0                                            
______________________________________________________________________________________________

We compile our model with Adam optimizer and lr=0.001 after various trials between optimizers and learning rates.

We chose categorical crossentropy as the loss function and accuracy as our metric as dirrected by our use case and standard practices.

We also split our training data and validation data in 80% and 20% ratio.

We kept a callback for checkpointing best performing models after each epoch.

In [0]:
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

history = model.fit([categorical_features,
                     numeric_features,
                     title,
                     domain,
                     url],label,validation_split=0.2, batch_size=16, epochs=10, callbacks=[tf.keras.callbacks.ModelCheckpoint(
    '{epoch}', monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch')])

Train on 473187 samples, validate on 118297 samples
Epoch 1/10
473184/473187 [============================>.] - ETA: 0s - loss: 1.9321 - accuracy: 0.6095
Epoch 00001: val_loss improved from inf to 1.77269, saving model to 1
INFO:tensorflow:Assets written to: 1/assets


INFO:tensorflow:Assets written to: 1/assets


473187/473187 [==============================] - 2627s 6ms/sample - loss: 1.9321 - accuracy: 0.6095 - val_loss: 1.7727 - val_accuracy: 0.7699
Epoch 2/10
433280/473187 [==========================>...] - ETA: 2:49 - loss: 1.9191 - accuracy: 0.6227Buffered data was truncated after reaching the output size limit.

Plotting our training and validation accuracies along with the training and validation losses.

This gives us a pretty clear picture if our model is underfitting or overfitting and track the problems.

In [0]:
print(history.history.keys())
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('mse')
plt.ylim([0,2.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Saving the final model for serving.

In [0]:
tf.keras.models.save_model(model,filepath='megav1_1.h5',save_format='h5')